# Cleaning Used Cars Dataset (craigslist)
source: https://www.kaggle.com/datasets/austinreese/craigslist-carstrucks-data?select=vehicles.csv

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [ ]:
dataset_path = './data/vehicles_craigslist.csv'
raw_df = pd.read_csv(dataset_path)

### Check features stats

In [ ]:
raw_df.describe(include='object')

#### Notes: 
    * some categories are unbalanced (transmission, title_status, fuel)
    * cylinders must be converted to number

In [ ]:
raw_df.describe()

### Remove unused columns

In [ ]:
cl_df = raw_df.drop(columns=['url', 'region', 
                    'region_url', 'title_status', 
                    'VIN', 'image_url', 'description', 
                    'state', 'posting_date',
                    'id', 'county'])

### Check types in dataset

In [ ]:
cl_df.dtypes

### Check outliers

In [ ]:
def draw_boxplots(df):
    # Set the style of seaborn
    sns.set_theme(style="whitegrid")
    
    # Select only numeric features
    numeric_features = df.select_dtypes(include=['number']).columns
    
    # Determine the number of rows and columns for the grid
    n_features = len(numeric_features)
    n_cols = 3  # Number of columns in the grid
    n_rows = int(np.ceil(n_features / n_cols))  # Calculate number of rows needed
    
    # Create a grid of boxplots
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows))  # Adjust the figure size as needed
    axes = axes.flatten()  # Flatten the axes array for easy indexing
    
    # Loop through each numeric feature and create a boxplot
    for i, column in enumerate(numeric_features):
        sns.boxplot(y=df[column], ax=axes[i])
        axes[i].set_title(column)
    
    # Hide any unused subplots
    for j in range(i + 1, n_rows * n_cols):
        fig.delaxes(axes[j])
    
    plt.tight_layout()  # Adjust layout to prevent overlap

    
def get_outliers_df(df):
    # Function to create a DataFrame with all rows that contain outliers
    outliers_df = pd.DataFrame()  # Initialize an empty DataFrame
    
    # Loop through each numerical feature
    for column in df.select_dtypes(include=['number']).columns:
        # Calculate Q1 (25th percentile) and Q3 (75th percentile)
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        
        # Calculate IQR
        IQR = Q3 - Q1
        
        # Determine lower and upper bounds
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Identify outliers
        outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
        
        # Append outliers to the outliers_df DataFrame
        outliers_df = pd.concat([outliers_df, outliers], ignore_index=True)
    
    # Drop duplicates if any row is an outlier for multiple columns
    outliers_df = outliers_df.drop_duplicates()
    
    return outliers_df


# Function to count outliers based on IQR
def count_outliers(df):
    outlier_counts = {}
    
    # Loop through each numerical feature
    for column in df.select_dtypes(include=['number']).columns:
        # Calculate Q1 (25th percentile) and Q3 (75th percentile)
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        
        # Calculate IQR
        IQR = Q3 - Q1
        
        # Determine lower and upper bounds
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Count outliers
        outlier_count = df[(df[column] < lower_bound) | (df[column] > upper_bound)].shape[0]
        outlier_counts[column] = outlier_count
    
    return outlier_counts

# Count outliers
outliers = count_outliers(cl_df)

 
def remove_outliers(df):
    # Function to remove outliers based on interquartile range (IQR) method
    # Create a copy of the DataFrame to avoid modifying the original
    df_cleaned = df.copy()
    
    # Loop through each numerical feature
    for column in df_cleaned.select_dtypes(include=['number']).columns:
        # Calculate Q1 (25th percentile) and Q3 (75th percentile)
        Q1 = df_cleaned[column].quantile(0.25)
        Q3 = df_cleaned[column].quantile(0.75)
        
        # Calculate IQR
        IQR = Q3 - Q1
        
        # Determine lower and upper bounds
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Filter the DataFrame to remove outliers
        df_cleaned = df_cleaned[(df_cleaned[column] >= lower_bound) & (df_cleaned[column] <= upper_bound)]
    
    return df_cleaned

# Display the number of outliers for each feature
print("Number of outliers in each feature:")
for feature, count in outliers.items():
    print(f"{feature}: {count}")


def display_dataframe(df):
    # Create a Plotly table
    fig = go.Figure(data=[go.Table(
        header=dict(values=list(df.columns),
                    fill_color='paleturquoise',
                    align='left'),
        cells=dict(values=[df[col] for col in df.columns],
                fill_color='lavender',
                align='left'))
    ])
    # Update layout and show the figure
    fig.update_layout(title='DataFrame')
    fig.show();

In [ ]:
print(cl_df.shape)

In [ ]:
draw_boxplots(cl_df)

In [ ]:
outliers_df = get_outliers_df(cl_df)
display_dataframe(outliers_df)

### Remove Outliers

In [ ]:
cl_df = remove_outliers(cl_df)
print(cl_df.shape)
draw_boxplots(cl_df)

### Check NaNs

In [ ]:
cl_df.isna().sum().sort_values(ascending=False)

### Add unknown values for missing data

In [ ]:

def add_unknown_values(df):
    # Create a copy of the DataFrame to avoid modifying the original
    df_filled = df.copy()
    
    # Fill missing values with 'unknown' for categorical features
    for column in df_filled.select_dtypes(include=['object', 'category']).columns:
        df_filled[column] = df_filled[column].fillna('unknown')
    
    return df_filled

df_filled = add_unknown_values(cl_df)
print(df_filled.isna().sum().sort_values(ascending=False))

### Show counts of categorical variables

In [ ]:
def plot_category_counts(df):
    categorical = df.select_dtypes(include=['object', 'category']).columns
    # Filter out the 'model' column
    categorical = [col for col in categorical if col != 'model']
    
    n_cols = 2  # Number of columns
    n_rows = (len(categorical) + n_cols - 1) // n_cols  # Calculate number of rows needed

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(8 * n_cols, 6 * n_rows))  # Create a grid of subplots
    axes = axes.flatten()  # Flatten the axes array for easy indexing

    for i, col in enumerate(categorical):
        counts = df[col].value_counts().sort_values(ascending=False)
        sns.countplot(data=df, y=col, palette='viridis', hue=col, legend=False, order=counts.index, ax=axes[i])
        axes[i].set_title(f'Counts of Each Category in {col}')
        axes[i].set_ylabel('Category')
        axes[i].set_xlabel('Count')

    # Hide any unused subplots
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')

    plt.tight_layout()
    plt.show()


In [ ]:
plot_category_counts(df_filled)

In [ ]:
# Show counts for all categorical features
for col in cl_df.select_dtypes(include=['category', 'object']).columns:
    print(f"Counts for {col}:")
    print(cl_df[col].value_counts())
    print()

In [ ]:
### convert color features green, brown, custom, orange, yellow, and purple to 'other'
def convert_color_features(df):
    # Create a copy of the DataFrame to avoid modifying the original
    df_converted = df.copy()
    
    # Define the colors to convert
    colors_to_convert = ['green', 'brown', 'custom', 'orange', 'yellow', 'purple']
    
    # Convert specified colors to 'other'
    for color in colors_to_convert:
        df_converted.loc[df_converted['paint_color'] == color, 'paint_color'] = 'other'
    
    return df_converted

df_filled = convert_color_features(df_filled)

#### Drop size column because the majority of values are 'unknown

In [ ]:

df_filled = df_filled.drop(columns=['size'])

#### Remove rows with manufacturer harley-davidson (motorcycles)


In [ ]:
df_filled = df_filled[df_filled['manufacturer'] != 'harley-davidson']

## Create first random forest model to check feature importance

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
# First fast approach
print(len(df_filled['model'].unique()))

# Remove model column (there are too many different car models)
df_filled = df_filled.drop(columns=['model'])

#### Label encoding for condition and cylinders

In [ ]:

condition_mapping = {
    'unknown': 0,
    'salvage': 1,
    'fair': 2,
    'good': 3,
    'excellent': 4,
    'like new': 5,
    'new': 6
}
cylinders_mapping = {
    'unknown': 0,
    'other': 1,
    '3 cylinders': 2,
    '4 cylinders': 3,
    '5 cylinders': 4,
    '6 cylinders': 5,
    '8 cylinders': 6,
    '10 cylinders': 7,
    '12 cylinders': 8
}

df_filled['condition_encoded'] = df_filled['condition'].map(condition_mapping)
df_filled['cylinders_encoded'] = df_filled['cylinders'].map(cylinders_mapping)

#### Label encoding for manufacturer 

In [ ]:

from sklearn.preprocessing import LabelEncoder 
# Create a LabelEncoder instance
le = LabelEncoder()
# Fit and transform the 'manufacturer' column
df_filled['manufacturer_encoded'] = le.fit_transform(df_filled['manufacturer'])

In [ ]:
display_dataframe(df_filled)

In [ ]:
# Remove cylinders, condition and manufacturer columns
df_filled = df_filled.drop(columns=['cylinders', 'condition', 'manufacturer'])

In [ ]:
df_filled.dtypes

### One hot encoding for the other categorical features

In [ ]:
# Identify categorical columns
categorical_cols = df_filled.select_dtypes(include=['object', 'category']).columns

In [ ]:
# Convert categorical columns to one-hot encoding
encoded_df = pd.get_dummies(df_filled, columns=categorical_cols)

In [ ]:
print(encoded_df.shape)

In [ ]:
# Split the dataset into features and target
X = encoded_df.drop(columns=['price'])
y = encoded_df['price']

# Perform stratified split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=X, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Train the Random Forest model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

In [ ]:
# Make predictions
y_pred = model.predict(X_test)

# Calculate errors
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f'MAE: {mae}, MSE: {mse}, RMSE: {rmse}')

In [ ]:
results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

In [ ]:
results.describe()

In [ ]:
def plot_results(df_results):
    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=df_results, x='Actual', y='Predicted', color='blue', label='Predictions')
    sns.lineplot(x=[df_results['Actual'].min(), df_results['Actual'].max()], 
                 y=[df_results['Actual'].min(), df_results['Actual'].max()], 
                 color='red', label='Perfect Prediction', linestyle='--')
    plt.title('Actual vs Predicted Values')
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.legend()
    plt.show()
    
plot_results(results)

In [ ]:
# Get feature importances
importances = model.feature_importances_

feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

In [ ]:
# Plot the feature importances with Seaborn
plt.figure(figsize=(5, 10))
sns.barplot(data=feature_importance_df, x='Importance', y='Feature', hue='Feature', palette='viridis', legend=False)
plt.title('Feature Importances from Random Forest (with One-Hot Encoding)')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show();

In [ ]:
plt.figure(figsize=(20, 10))
n_points = 300
sns.lineplot(x=range(n_points), y=results['Predicted'][0:n_points], color='blue', label='predicted')
sns.lineplot(x=range(n_points), y=results['Actual'][0:n_points], color='red', label='Actual')